In [ ]:
! pip install transformers

import numpy as np
import pandas as pd

import torch
import sys
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler#, get_linear_schedule_with_warmup
#from pytorch_pretrained_bert import BertTokenizer, BertConfig
from sklearn.metrics import recall_score, precision_score
from tqdm import trange


from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup, AdamW

     |████████████████████████████████| 1.5MB 12.8MB/s 
     |████████████████████████████████| 890kB 57.4MB/s 
     |████████████████████████████████| 2.9MB 57.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b872e71e9029b27adf87c009871fadec96530efa127c12a252552bec6ab38a51
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r -f /content/drive/'My Drive'/msu/re_from_bioBert /content/
import sys

sys.path.append('/content/msu/re_from_bioBert/')
dir_name = '/content/re_from_bioBert/'

Mounted at /content/drive


In [ ]:
def fit(model, x, y, n_epochs, lr, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]
    
    t_inputs = torch.tensor(x)
    t_labels = torch.tensor(y)
    t_masks = torch.tensor(masks)

    train_data = TensorDataset(t_inputs, t_masks, t_labels)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.cuda()

    param_optimizer = list(model.named_parameters())
    no_decay  = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
    
    optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=lr,
    eps=1e-6)

    epochs = n_epochs
    max_grad_norm = 1.5

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.


    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0.1 * total_steps,
        num_training_steps=total_steps)
    
    model.train()
    for _ in trange(epochs, desc="Epoch"):
        total_loss = 0

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            model.zero_grad()
        
            outputs = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
        
            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()

            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            optimizer.step()
            scheduler.step()
        print(' Loss: ', total_loss)    


def predict(model, x, batch_size=32):
    masks = [[float(i != 0.0) for i in ii] for ii in x]

    t_inputs = torch.tensor(x)
    t_masks = torch.tensor(masks)

    test_data = TensorDataset(t_inputs, t_masks)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions = []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        outputs = outputs[0].detach().cpu().numpy()
        
        batch_labels = np.argmax(outputs, axis=1).tolist()
        predictions += batch_labels
    return predictions

def get_data_from_df(df, tokenizer):
    lbls, raw_txt = df.lbl.values, df.txt.values
    txt = ['[CLS] ' + i + ' [SEP]' for i in raw_txt]
    res = []
    for sent in txt:
      tokens = tokenizer.tokenize(sent)
      tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
      while len(tokens_ids) < MAX_LEN:
        tokens_ids.append(0)
      res.append(tokens_ids)
    return res, lbls
    

def cv_iteration(model, tokenizer, data_dir, cur_it, n_epochs, lr, batch_size=32):
    train_df = pd.read_csv(data_dir + '/'+ str(cur_it+1) + '/train.tsv', sep = '\t', names = ['txt', 'lbl'])
    train_tokens, train_lbls = get_data_from_df(train_df, tokenizer)
    train_lbls = [int(i) for i in train_lbls]
    
    test_df = pd.read_csv(data_dir + '/'+ str(cur_it+1) + '/test.tsv', sep = '\t', names = ['txt', 'lbl'])
    test_tokens, test_lbls = get_data_from_df(test_df, tokenizer)
    test_tokens, test_lbls = test_tokens[1:], test_lbls[1:]
    test_lbls = [int(i) for i in test_lbls]
    
    
    fit(model, train_tokens, train_lbls, n_epochs, lr, batch_size)
    preds = predict(model, test_tokens, batch_size)
    #print(np.bincount(np.array(train_tokens)), np.bincount(np.array(preds)))

    precision, recall = precision_score(test_lbls, preds), recall_score(test_lbls, preds)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1


def get_cv_scores(model_name, tokenizer, data_dir, lr,  n_epochs, batch_size=32, st_from=0):
    prescision_s, recall_s, f1_s = [], [], []
    for i in range(st_from, 10):
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        p, r, f = cv_iteration(model, tokenizer, data_dir, i, n_epochs, lr, batch_size)
        print('P: ', p, 'R: ', r, 'F1: ', f)
        prescision_s.append(p)
        recall_s.append(r)
        f1_s.append(f)
    return sum(prescision_s) / len(prescision_s), sum(recall_s) / len(recall_s), sum(f1_s) / len(f1_s) 


In [ ]:
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = "dmis-lab/biobert-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 3, 32)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:  33%|███▎      | 1/3 [02:02<04:04, 122.36s/it]

 Loss:  91.39673164486885


Epoch:  67%|██████▋   | 2/3 [04:09<02:03, 123.77s/it]

 Loss:  68.3507828116417


Epoch: 100%|██████████| 3/3 [06:17<00:00, 125.67s/it]

...

Epoch:  67%|██████▋   | 2/3 [04:15<02:07, 127.59s/it]

 Loss:  66.50431564450264


Epoch: 100%|██████████| 3/3 [06:22<00:00, 127.46s/it]

 Loss:  52.94078643620014


P:  0.7155172413793104 R:  0.8892857142857142 F1:  0.7929936305732483
P:  0.7499030628844078 R:  0.8943124046771734 F1:  0.8153220493148933


In [ ]:
MAX_LEN = 260

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = "dmis-lab/biobert-v1.1"
data_dir = "/content/re_from_bioBert/euadr"
res = get_cv_scores(model, tokenizer, data_dir, 3e-5, 3, 32)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:  33%|███▎      | 1/3 [00:15<00:30, 15.33s/it]

 Loss:  5.956295996904373


Epoch:  67%|██████▋   | 2/3 [00:31<00:15, 15.49s/it]

 Loss:  5.452409356832504


Epoch: 100%|██████████| 3/3 [00:46<00:00, 15.47s/it]

...

Epoch:  67%|██████▋   | 2/3 [00:30<00:15, 15.11s/it]

 Loss:  5.740025669336319


Epoch: 100%|██████████| 3/3 [00:45<00:00, 15.17s/it]

 Loss:  5.624357432126999


P:  0.7428571428571429 R:  1.0 F1:  0.8524590163934427
P:  0.7682547482547483 R:  0.9653846153846153 F1:  0.8495608181834833


In [ ]:
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
model = "dmis-lab/biobert-large-cased-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 3, 16)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

 Loss:  175.4302276968956


Epoch:  67%|██████▋   | 2/3 [14:00<06:57, 417.47s/it]

 Loss:  125.84052204340696


Epoch: 100%|██████████| 3/3 [21:09<00:00, 423.03s/it]

...

Epoch:  67%|██████▋   | 2/3 [14:06<07:03, 423.08s/it]

 Loss:  211.26532024145126


Epoch: 100%|██████████| 3/3 [21:10<00:00, 423.63s/it]

 Loss:  210.27541077136993


P:  0.525328330206379 R:  1.0 F1:  0.6888068880688807


In [ ]:
MAX_LEN = 150

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
model = "dmis-lab/biobert-large-cased-v1.1"
data_dir = "/content/re_from_bioBert/GAD"
res = get_cv_scores(model, tokenizer, data_dir, 2e-5, 3, 16, 8)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

 Loss:  185.6430408656597


Epoch:  67%|██████▋   | 2/3 [07:57<03:58, 238.75s/it]

 Loss:  136.77046521008015


Epoch: 100%|██████████| 3/3 [11:56<00:00, 238.76s/it]

...

Epoch:  67%|██████▋   | 2/3 [07:57<03:58, 238.90s/it]

 Loss:  130.93174132704735


Epoch: 100%|██████████| 3/3 [11:56<00:00, 238.88s/it]

 Loss:  88.90952000766993


P:  0.7373134328358208 R:  0.8821428571428571 F1:  0.8032520325203252
P:  0.7381581826935703 R:  0.8910714285714285 F1:  0.807423117709438


In [ ]:
MAX_LEN = 260

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
model = "dmis-lab/biobert-large-cased-v1.1"
data_dir = "/content/re_from_bioBert/euadr"
res = get_cv_scores(model, tokenizer, data_dir, 3e-5, 3, 8)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

 Loss:  22.661943286657333


Epoch:  67%|██████▋   | 2/3 [01:01<00:30, 30.55s/it]

 Loss:  19.194591268897057


Epoch: 100%|██████████| 3/3 [01:31<00:00, 30.50s/it]

...

Epoch:  67%|██████▋   | 2/3 [01:01<00:30, 30.66s/it]

 Loss:  24.994070649147034


Epoch: 100%|██████████| 3/3 [01:31<00:00, 30.63s/it]

 Loss:  23.77052417397499


P:  0.7428571428571429 R:  1.0 F1:  0.8524590163934427


In [ ]:
MAX_LEN = 260

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
model = "dmis-lab/biobert-large-cased-v1.1"
data_dir = "/content/re_from_bioBert/euadr"
res = get_cv_scores(model, tokenizer, data_dir, 3e-5, 3, 8, 5)
p, r, f = res
print('P: ', p, 'R: ', r, 'F1: ', f)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

 Loss:  25.82494419813156


Epoch:  67%|██████▋   | 2/3 [01:01<00:30, 30.71s/it]

 Loss:  23.961319237947464


Epoch: 100%|██████████| 3/3 [01:31<00:00, 30.65s/it]

...

Epoch:  67%|██████▋   | 2/3 [01:01<00:30, 30.59s/it]

 Loss:  20.78427827358246


Epoch: 100%|██████████| 3/3 [01:31<00:00, 30.58s/it]

 Loss:  17.266600236296654


P:  0.7428571428571429 R:  1.0 F1:  0.8524590163934427
P:  0.7481318681318682 R:  0.9538461538461538 F1:  0.835813366960908
